# Title: Different result using same calculations for Python and Julia.

We are having the below code blocks:

For Python:

```python
import numpy as np
import matplotlib.pyplot as plt

# Set the seed for reproducibility
# np.random.seed(1234)

mu = 2
sigma = 0.1

Xzero = 1

T = 1
N = 2**9
dt = T/N

reps = 10**5
decreasing_number = 5

dW = np.sqrt(dt) * np.random.randn(reps, N)
W = np.cumsum(dW, axis=1)

Xpath = np.zeros((reps, N+1))
Xpath[:, 0] = Xzero

Xem = {}

for i in range(1, decreasing_number + 1):
    Xem[i - 1] = np.zeros((reps, (N//(2**(i-1)))+1))

for i in range(N):
    Xpath[:, i + 1] = Xpath[:, i] * np.exp((mu - 0.5*sigma**2)*dt + sigma*dW[:, i])

for i in range(1, decreasing_number + 1):
    Xem[i - 1][:, 0] = Xzero

    dWTemp = np.zeros((reps, N//2**(i-1)))

    for j in range(2**(i-1)):
        dWTemp += dW[:, j::2**(i-1)]

    for j in range(N//2**(i-1)):
        Xem[i - 1][:, j + 1] = Xem[i - 1][:, j] + mu*Xem[i - 1][:, j]*dt*2**(i-1) + sigma*Xem[i - 1][:, j]*dWTemp[:, j]

expectedPathEnd = np.mean(Xpath[:, -1])
error = np.zeros(decreasing_number)

for i in range(1, decreasing_number + 1):
    error[i-1] = abs(np.mean(Xem[i - 1][:, -1]) - expectedPathEnd)

log2_error = np.log2(error)

print(log2_error)

# The result is array([-5.12216237, -4.12863162, -3.14147915, -2.16727841, -1.21713823])
```

And for Julia:

```Julia
using Random, CairoMakie, Statistics, Distributions

# Random.seed!(1234)

mu = 2
sigma = 0.1

Xzero = 1

T = 1
N = 2^9
dt = T/N

reps = 10^3
decreasing_number = 5


dW = sqrt(dt) * randn(BigFloat, (reps, N))
# dW = sqrt(dt) * rand(Uniform(-1, 1), reps, N)
# dW = sqrt(dt) * rand(Normal(0, 1), reps, N)
W = cumsum(dW, dims = 2)

Xpath = zeros(BigFloat, reps, N+1)
Xpath[:, 1] .= Xzero

Xem = Dict()

for i in 1:decreasing_number
    Xem[i - 1] = zeros(BigFloat, reps, N÷2^(i-1)+1)
end

for i in 1:N
    Xpath[:, i + 1] .= Xpath[:, i].*exp.((mu .- 0.5*sigma^2)*dt .+ sigma*dW[:, i])
end

for i in 1:decreasing_number
    Xem[i - 1][:, 1] .= Xzero

    dWTemp = zeros(reps, N÷2^(i-1))

    for j in 0:2^(i-1)-1
        dWTemp .+= dW[:, j+1:2^(i-1):end]
    end

    for j in 1:N÷2^(i-1)
        Xem[i - 1][:, j + 1] .= Xem[i - 1][:, j] .+ mu*Xem[i - 1][:, j]*dt*2^(i-1) .+ sigma*Xem[i - 1][:, j].*dWTemp[:, j]
    end
end

expectedPathEnd = mean(Xpath[:, end])
error = zeros(BigFloat, decreasing_number)

for i in 1:decreasing_number
    error[i] = abs(mean(Xem[i - 1][:, end]) - expectedPathEnd)
end

log2_error = log.(error)

print(log2_error)

# The result is:
# 5-element Vector{BigFloat}:
#  -3.554214402096594687743060305349970824588774138976971536723727823461683147052966
#  -2.862899579149310574925670462201169384031459470310215567569001669928305681236022
#  -2.178214170645856020469139263167143618313205681041395430066430107576287576669499
#  -1.501862963526146322712563759336474109500760939648390401298967910116891895820844
#  -0.843632167676772655656675830018306695331363346180138261659490194335827273763738
```

When doing linear regression with the results, Python's result gives the slope of $\approx 0.977$ and Julia's results gives the slope of $\approx 0.678$. The case of Julia is not consistent with the theoretical result (slope $1$) whereas Python is. I suspect the problem lies in the random number generator of Julia vs Python?